# tornado中实现异步非阻塞

在Tornado中，现在有两种方式实现协程：
1. 函数前加@tornado.gen.coroutine修饰器，异步函数前加yield，这是一种较“老”的方式；
2. 函数采用async定义，异步函数前加await。async和await是python3.5引入的，目前是python内置方式，因此后者的运行效率比前者要高些。

但Tornado官网上也说了后者的不足：对老版本的兼容不如前者；yield适用性上也比await广些，例如面对future list时。

### 使用gen.coroutine和run_on_executor方式

In [ ]:
import time
import json
import tornado.web
import tornado.gen
import tornado.ioloop
from tornado.concurrent import run_on_executor
from concurrent.futures import ThreadPoolExecutor


class MainHandler(tornado.web.RequestHandler):
    executor = ThreadPoolExecutor(200)

    @tornado.gen.coroutine
    def get(self):
        kw = self.get_argument("kw", None)
        if kw is None:
            return self.write(json.dumps([]))
        reback_list = yield self.sleep(kw)
        self.write(json.dumps(reback_list))
        self.finish()

    @run_on_executor
    def sleep(self, kw):
        kw = int(kw)
        time.sleep(kw)
        return {"aaa": "aaa"}


def start():
    application = tornado.web.Application([
            (r"/", MainHandler),
        ])
    application.listen(8001)
    tornado.ioloop.IOLoop.current().start()


if __name__ == "__main__":
    start()

该方式是通过多线程来实现异步非阻塞,是一种比较老的方式

### 使用async和await方式

In [ ]:
import time
import json
import tornado.web
import tornado.ioloop


class BaseHandler(tornado.web.RequestHandler):
    
    def async_call(self, func, *args):
        return tornado.ioloop.IOLoop.current().run_in_executor(None, func, *args)

class MainHandler(BaseHandler):

    aysnc def get(self):
        kw = self.get_argument("kw", None)
        if kw is None:
            return self.write(json.dumps([]))
        reback_list = await self.async_call(self.sleep, kw)
        self.write(json.dumps(reback_list))
        self.finish()

    def sleep(self, kw):
        kw = int(kw)
        time.sleep(kw)
        return {"aaa": "aaa"}


def start():
    application = tornado.web.Application([
            (r"/", MainHandler),
        ])
    application.listen(8001)
    tornado.ioloop.IOLoop.current().start()


if __name__ == "__main__":
    start()

`tornado.ioloop.IOLoop.current().run_in_executor`运行。`run_in_executo`的定义如下：`def run_in_executor(self, executor, func, *args)`
executor是func的执行器，它应是`concurrent.futures.Executor`的子类，如果executor未指定，IOLoop将创建了一个线程池，最大线程数是cpu数的5倍：
`self._executor = ThreadPoolExecutor(max_workers=(cpu_count() * 5))`,因此，阻塞函数依然是按阻塞方式运行，只不过在另一线程中（同时运行的任务不能多于最大线程数，否则也要阻塞），看起来就变成非阻塞了。

上面的例子比较简单，所以看起来异步操作也不麻烦（未涉及锁），Tornado也提供了必要的封装，但业务比较复杂时，写一个全部都是异步操作的web应用真不是件容易的事情，要求我们的思路要“跳跃”，相对比较难设计与调试。其实，多线程执行多任务运行并非一无是处，只是在任务很多时，效率不佳。

如果不采用异步处理，而又要提高效率，就必须采用下面描述的“多进程部署”了，此时，编程结构就“自然”多了。